In [1]:
#导入需要用到的包
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt

import warnings 
from numpy import *

warnings.filterwarnings('ignore') #忽略匹配的警告
matplotlib.rc("font", family='Kaiti')#设置中文字体
matplotlib.rcParams['axes.unicode_minus'] = False#正确显示正负号

In [116]:
close = df_close
close_pre = close.shift(1)
Return = (close - close_pre) / close_pre

factor = df_factor_1

ltg = df_FM
FloatMarketValue = close * ltg

close = close.loc[pd.to_datetime('2023-06-01'):pd.to_datetime('2023-12-25')]
close_pre = close_pre.loc[pd.to_datetime('2023-06-01'):pd.to_datetime('2023-12-25')]
Return = Return.loc[pd.to_datetime('2023-06-01'):pd.to_datetime('2023-12-25')]
all_date = close.index.strftime('%Y-%m-%d').tolist()
all_code = close.columns.tolist()

all_trade_dates = pd.DataFrame({'TradeDates':all_date},index=all_date)
all_trade_dates['Month'] = pd.to_datetime(all_trade_dates['TradeDates']).dt.strftime('%Y-%m')
all_trade_dates = all_trade_dates.groupby('Month')['TradeDates'].first().reset_index()['TradeDates'].tolist()
all_previous_trade_dates = [all_date[all_date.index(x)-1] for x in all_trade_dates]

In [115]:
#建立参数集合
param = {}

#设置回测参数
param['start_date'] = '2023-06-01' #回测开始时间
param['end_date'] = '2023-12-25' #回测结束时间
param['layer_num'] = 5 #回测分组数
param['fee'] = 0.002 #交易费用
#可选项
param['market_value_neutral'] = True  # 市值中性化
param['print'] = True  # 打印日志


## 数据处理
class Get_Data():

    def __init__(self,param,df_close,df_FM,df_factor):
       
        self.param = param
        self.close = df_close  #获取收盘价
        self.close_pre = self.close.shift(1) #获取前一日收盘价

        # 获取因子
        self.factor = df_factor

        # 计算流通市值
        self.ltg = df_FM
        self.FloatMarketValue = self.close * self.ltg

        # 计算收益率序列
        self.Return = (self.close - self.close_pre) / self.close_pre

        # 截取开始与终止回测时间
        self.close = self.close.loc[pd.to_datetime(self.param['start_date']):pd.to_datetime(self.param['end_date'])]#截取回测时期内收盘价数据
        self.close_pre = self.close_pre.loc[pd.to_datetime(self.param['start_date']):pd.to_datetime(self.param['end_date'])]
        self.Return = self.Return.loc[pd.to_datetime(self.param['start_date']):pd.to_datetime(self.param['end_date'])]
        self.FloatMarketValue = self.FloatMarketValue.loc[pd.to_datetime(self.param['start_date']):pd.to_datetime(self.param['end_date'])]

        # 获取全部交易日
        self.all_dates = self.close.index.strftime('%Y-%m-%d').tolist()
        # 获取股票代码
        self.all_codes = self.close.columns.tolist()

        # 计算月初交易日换仓日序列
        self.all_trade_dates = pd.DataFrame({'TradeDates':self.all_dates},index=self.all_dates)
        self.all_trade_dates['Month'] = pd.to_datetime(self.all_trade_dates['TradeDates']).dt.strftime('%Y-%m')
        self.all_trade_dates = self.all_trade_dates.groupby('Month')['TradeDates'].first().reset_index()['TradeDates'].tolist()
        self.all_trade_dates = self.all_trade_dates[1:]  # 第一个月初始化持仓即可 不需要换仓
        # 计算月初交易日换仓日序列的上一个月底截止日
        self.all_previous_trade_dates = [self.all_dates[self.all_dates.index(x)-1] for x in self.all_trade_dates]
        
        # 进行因子预处理
        self.factor= self.preprocess(self.factor)
        
    def preprocess(self, factor):
        # 去除没有市值或者没有收盘价的股票
        factor[(self.FloatMarketValue.isna())==True] = np.nan
        factor[(self.close.isna())==True] = np.nan
        
        # 在每个日期截面上对所有股票进行因子去极值中性化标准化
        preprocessed_factor = []
        for i in self.all_previous_trade_dates:

            # 截取当日因子值
            temp_factor = factor.loc[i].to_frame()

            # 中位数分位点上去极值
            temp_factor_m = temp_factor.median().values[0]
            temp_factor_m1 = (temp_factor - temp_factor.median()).abs().mean().values[0]
            temp_factor.loc[temp_factor.iloc[:,0] > temp_factor_m+3*1.4826*temp_factor_m1] = temp_factor_m+3*1.4826*temp_factor_m1
            temp_factor.loc[temp_factor.iloc[:,0] > temp_factor_m+3*1.4826*temp_factor_m1] = temp_factor_m-3*1.4826*temp_factor_m1
            
            #因子中性化
            if self.param['market_value_neutral']==True:
                # 进行对数流通市值中性化
                temp_log_FloatMarketValue = np.log(self.FloatMarketValue.loc[i].to_frame())
                # 以对数流通市值中性化进行线性回归
                model = sm.OLS(temp_factor,temp_log_FloatMarketValue,missing='drop')
                results = model.fit()
                # 取残差为中性化后的因子
                temp_factor = (temp_factor.loc[results.fittedvalues.index] - results.fittedvalues.to_frame().rename(columns={0:pd.to_datetime(i)}))
                temp_factor = temp_factor.reindex(temp_factor.index)

            # 因子标准化
            temp_factor = temp_factor.replace(np.inf,np.nan).replace(-np.inf,np.nan)
            temp_factor = (temp_factor - temp_factor.mean()) / temp_factor.std()

            # 输出
            preprocessed_factor.append(temp_factor)

        # 合并处理完的factor因子值
        preprocessed_factor = pd.concat(preprocessed_factor,axis=1).reindex(self.all_codes)
        
        return preprocessed_factor


In [134]:
# 回测框架
class Backtest():
    
    # 初始化
    def __init__(self, param, gn_data):
        
        # 参数
        self.param = param
        
        # 回测参数
        self.layer_num = self.param['layer_num']
        self.fee = self.param['fee']
        
        # 传入数据
        self.close = gn_data.close
        self.all_dates = gn_data.all_dates
        self.all_codes = gn_data.all_codes
        self.all_trade_dates = gn_data.all_trade_dates
        self.all_previous_trade_dates = gn_data.all_previous_trade_dates
        
        # 因子标的
        self.factor = gn_data.factor
        # 计算因子分层
        self.df_stock_layer = self.generate_stock_layers()
        # 进行回测
        self.layer_backtest = self.Backtest()
        
    # 进行因子分层
    def generate_stock_layers(self):
        
        # 由于是月度调仓，所以需要先截取所有的月末序列 再换到月初调仓
        self.factor = self.factor[pd.to_datetime(self.all_previous_trade_dates)]  # self.all_previous_trade_dates是在General_Data计算好的所有月末序列
        self.factor.columns = self.all_trade_dates  # self.all_trade_dates是在General_Data计算好的所有月末序列对应的月初序列
        
        # 计算每一层因子持仓
        # df_stock_layer是一个字典，keys代表分层层数，values代表给定层数的因子持仓权重dataframe
        # 和self.factor不要混淆 是需要被分层持仓的因子dataframe
        df_stock_layer = dict()

        # 对回测标的因子进行日期循环
        for k, date in enumerate(self.factor.columns):
            
            # 截取当前日期的数据
            temp_factor_series_for_current_date = self.factor.loc[:, date].dropna()

            # 每层持股个数
            temp_stock_each_layer = int(len(temp_factor_series_for_current_date) / self.layer_num)
            # 每层每股权重
            temp_weight_each_layer = 1 / temp_stock_each_layer

            # 对每一层因子分层循环
            for i in range(self.layer_num):

                # 计算本层股票代码列表
                stock_list_for_layer_i = temp_factor_series_for_current_date.sort_values(ascending=False).iloc[temp_stock_each_layer * i: temp_stock_each_layer * (i+1)].index.tolist()
                # 如果是第一个交易日期（k是日期列的索引）
                if k == 0:
                    df_stock_layer[i] = pd.DataFrame({date: temp_weight_each_layer}, index=stock_list_for_layer_i).reindex(self.all_codes)
                # 如果不是第一个交易日期 就在第i层因子分层上面新增一列日期
                else:
                    df_stock_layer[i][date] = pd.Series(temp_weight_each_layer, index=stock_list_for_layer_i).reindex(self.all_codes)
        
        # 返回结果
        self.df_stock_layer = df_stock_layer
        return self.df_stock_layer
    
    # 实际回测
    def Backtest(self):
        
        # 初始化净值结果矩阵
        self.layer_backtest = pd.DataFrame()
        self.turnover = pd.DataFrame()

        # 对因子分层层数进行循环
        for i in range(self.layer_num):

            # 月初换仓日在所有交易日序列中的index
            id_dates_trade = [i_date for i_date,date in enumerate(self.all_dates) if date in self.all_trade_dates]

    
            value_daily = pd.Series(0, index=self.all_dates)
            # 开始回测到第一个换仓日前，由于没有交易信号，全部置为0
            value_daily.iloc[:id_dates_trade[0]+1] = 1 

            # 初始化因子权重，长度为所有股票数量
            weight_last = pd.Series(0, index=self.factor.index)

            # 初始化换手率（计算手续费）index为换仓日期
            turnover = pd.Series(0, index=self.all_trade_dates)

            # 实际权重（df_stock_layer是generate_stock_layers里面计算的一个字典，keys代表分层层数，values代表给定层数的因子持仓权重dataframe）
            # 行为所有股票，列为换仓日
            real_weight = pd.DataFrame(index=self.df_stock_layer[i].index, columns=self.df_stock_layer[i].columns)

            # 对交易日进行循环
            # 这里的id_dates_trade是换仓日在全部交易日中的index序列
            # i_date_trade代表这是换仓日序列中第几个换仓日 id_date_trade代表这个换仓日是交易日中第几个交易日
            # 都是index索引 不牵涉具体日期的值
            for i_date_trade, id_date_trade in enumerate(id_dates_trade): 
                # id_dates_trade例[20, 41, 64, 84, 101, 123]
                # i_dates_trade 如[0, 1, 2, 3, 4, 5]

                # ———————————————调仓设置（只调仓+记录，不更新净值）—————————————————

                # 确定权重
                real_weight.iloc[:,i_date_trade]  = self.df_stock_layer[i].iloc[:,i_date_trade].fillna(0).values

                # 记录上个月月末的权重和这个月月初的买入价
                # 月度  当前换仓日的weight序列，index为all_codes
                weight_now = real_weight.iloc[:, i_date_trade]
                # 日度 换仓日close序列, index为all_codes  
                price_buy = self.close.T.iloc[:, id_date_trade] 

                # 记录当前基准value的值（是上一个月末延续下来的净值日，应该记录）
                value_port = value_daily.iloc[id_date_trade]  # todo
                # 记录换手率（双边换手率）
                turnover.iloc[i_date_trade] = sum(abs(weight_now.values - weight_last.values))
                value_port = value_port * (1 - turnover.iloc[i_date_trade] * self.fee)  # 扣交易费用

                # ——————————————月内日度净值更新（只更新净值，不调仓）—————————————————
                # 在月内日度循环更新净值前，应该先确定自己更新的日期索引区间（有回测第一个净值日前，和回测最后一个净值日后两个特殊点）
                # 如果不是最后一个换仓日 指当前日期索引小于换仓日日期在全部交易日日期索引中最后一位
                if id_date_trade < id_dates_trade[-1]:
                    # 净值日为换仓日至下一个换仓日
                    # 换仓日的下一日 至 下一个换仓日之间的all_codes的索引
                    id_dates_value = list(range(id_date_trade+1, id_dates_trade[i_date_trade + 1] + 1))
                # 如果是最后一个换仓日
                elif id_date_trade == id_dates_trade[-1]:
                    # 净值日为换仓日至回测终止日
                    id_dates_value = list(range(id_date_trade+1, len(self.all_dates)))
                else:
                    id_dates_value = []

                # 计算每日净值（实现月度调仓内部的日度净值更新问题）
                for id_date_value in id_dates_value:
                    # id_dates_value -> [] 1个换仓周期的日期索引
                    # 第一个日期为换仓日的第二天，最后一个日期为下一个换仓日当天
                    # 例换仓日index [20,25,30], 则可能为 [21,22,23,24,25]

                    # 记录后复权价
                    # 当前日收盘价，index为all_codes
                    price_value = self.close.T.iloc[:, id_date_value]

                    # 如果是月内最后一个净值日
                    if id_date_value == id_dates_value[-1]:
                        # 计算自然增长的权重，用于计算换手率和交易费用
                        # 现在的实际权重等于上个月的月末权重 * 月底调仓买入价 / 月初调仓卖出价
                        weight_last = (weight_now * price_value / price_buy).fillna(0)
                        if self.param['print']:
                            print('\n月末调仓！最新权重和:{}'.format(weight_last.sum()))

                    # 计算收益率
                    returns_port = np.nansum(weight_now * (price_value / price_buy - 1))     #收益=sum（上月末权重*（收盘价/买入价-1）） 

                    # 计算净值；若为当期换仓日的最后一个净值日，则此净值为下期换仓日计算净值的基准
                    value_daily.iloc[id_date_value] = value_port * (1 + returns_port)  #当期净值=上期净值*（1+收益）

                    if self.param['print']:
                        # 输出日志
                        print('{}  收益{:.2%}，累计净值{:.4}'.format(self.all_dates[id_date_value],
                                                       returns_port,
                                                       value_daily.iloc[id_date_value]))

            # 输出结果
            print('分层{}回测完成'.format(i+1))
            # 保存净值曲线，换手率等指标
            self.layer_backtest['group{}'.format(i+1)] = value_daily.dropna()
            self.turnover['group{}'.format(i+1)] = turnover.dropna()
        
        
        # 输出结果
        return self.layer_backtest

In [135]:
df = pd.read_csv('../量化data/df_hs.csv',index_col=0)
df['date'] = pd.to_datetime(df['date'])
df_close = df.pivot(index='date', columns='stock_code', values='close')
df_FM = df.pivot(index='date', columns='stock_code', values='流通股')

df_factor_1 = pd.read_csv('../量化data/alpha158/BETA10.csv')
df_factor_1['datetime'] = pd.to_datetime(df_factor_1['datetime'])
df_factor_1.set_index(['datetime'],inplace=True)

gn = Get_Data(param,df_close,df_FM,df_factor_1)   
backtest = Backtest(param,gn)

2023-07-04  收益-0.11%，累计净值0.9969
2023-07-05  收益-1.02%，累计净值0.9878
2023-07-06  收益-1.48%，累计净值0.9832
2023-07-07  收益-1.97%，累计净值0.9784
2023-07-10  收益-1.73%，累计净值0.9808
2023-07-11  收益-1.16%，累计净值0.9864
2023-07-12  收益-1.45%，累计净值0.9836
2023-07-13  收益-0.86%，累计净值0.9894
2023-07-14  收益-1.54%，累计净值0.9826
2023-07-17  收益-1.83%，累计净值0.9798
2023-07-18  收益-1.79%，累计净值0.9801
2023-07-19  收益-2.16%，累计净值0.9765
2023-07-20  收益-3.48%，累计净值0.9633
2023-07-21  收益-3.80%，累计净值0.9601
2023-07-24  收益-3.88%，累计净值0.9592
2023-07-25  收益-2.79%，累计净值0.9702
2023-07-26  收益-2.99%，累计净值0.9682
2023-07-27  收益-3.34%，累计净值0.9646
2023-07-28  收益-2.51%，累计净值0.9729
2023-07-31  收益-1.89%，累计净值0.9791

月末调仓！最新权重和:0.9791646007687737
2023-08-01  收益-2.08%，累计净值0.9772
2023-08-02  收益-0.15%，累计净值0.972
2023-08-03  收益1.42%，累计净值0.9873
2023-08-04  收益1.56%，累计净值0.9886
2023-08-07  收益0.08%，累计净值0.9743
2023-08-08  收益0.00%，累计净值0.9735
2023-08-09  收益-0.20%，累计净值0.9716
2023-08-10  收益0.20%，累计净值0.9754
2023-08-11  收益-2.53%，累计净值0.9489
2023-08-14  收益-3.60%，累计净值0.9385
2023-08-15  收益-

In [137]:
x = [i_date for i_date,date in enumerate(all_date) if date in all_trade_dates]

In [138]:
x

[0, 20, 41, 64, 84, 101, 123]

In [142]:
i_date_trade = 1
id_date_trade = 20
id_dates_trade = x
id_dates_value = list(range(id_date_trade+1, id_dates_trade[i_date_trade + 1] + 1))
id_dates_value

[21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41]

In [141]:
for i_date_trade, id_date_trade in enumerate(x):
    print( id_date_trade)

0
20
41
64
84
101
123


In [136]:
backtest.layer_backtest

,group1,group2,group3,group4,group5
2023-06-01,1.000000,1.000000,1.000000,1.000000,1.000000
2023-06-02,1.000000,1.000000,1.000000,1.000000,1.000000
2023-06-05,1.000000,1.000000,1.000000,1.000000,1.000000
2023-06-06,1.000000,1.000000,1.000000,1.000000,1.000000
2023-06-07,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...
2023-12-19,0.816771,0.838564,0.884666,0.853844,0.842612
2023-12-20,0.811849,0.832633,0.875130,0.839864,0.823295
2023-12-21,0.815699,0.836798,0.880816,0.849398,0.842379
2023-12-22,0.816293,0.839628,0.882314,0.848160,0.847039
